# Pull Forest Inventory Plot Coordinates and Areas

In [2]:
author = ['Marshall Worsham']
date = 2020-10-23

This notebook contains scripts that retrieve the coordinates from shapefiles marking the locations of 14 Kueppers 40x40m forest inventory plots in the East River watershed, Colorado, US, as of August 2020. The code operates on two types of shapefiles for each plot:

1. polygon shapefile representing the footprints of all plots, which contains computed Cartesian area and plot center coordinates
2. point shapefile representing the four corners of the plot

The script pulls the stored area and plot center coordinates from the polygon's data table. Then it imports the points into a geodataframe with CRS = WGS84 and pulls the coordinates from the imputed geometries of the four corners.

## 1. Load libraries

In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import math
import re
from matplotlib import pyplot as plt
from os.path import join, getsize
%matplotlib inline

## 2. Define paths and shapefile locations

In [114]:
# Define a few directories
os.getcwd()
directory = '/Volumes/GoogleDrive/My Drive/Research/RMBL/RMBL-East River Watershed Forest Data/Data/Geospatial/Kueppers_EastRiver_Plot_Shapefiles_WGS84UTM13N/'
#source_dir = os.sep.join([directory, 'Source'])
#out_dir = os.sep.join([, 'Output'])
out_dir = '/Users/hmworsham/Desktop'

# Call a directory storing all shapefiles and list contents
#sf_dir = os.sep.join([directory, 'Scratch/Kueppers_Plot_Bnd_2020_WGS84UTM13N_Renamed'])
#os.listdir(sf_dir)[:10]
sf_dir = directory
os.listdir(sf_dir)

['Polygons',
 'Lines',
 'Centroid_Points',
 'Corner_Points',
 'AllPlots',
 'Kueppers_EastRiver_AllPlots_2021_WGS84UTM13N.zip',
 'Icon\r']

## 3. Pull coordinates from the 'All Plots' shapefile

In [45]:
# Isolate the polygon sf containing all plot footprints
allfiles = [i for d,s,i in os.walk(sf_dir)]
alldirs = [s for d,s,i in os.walk(sf_dir)]
apdir = [s for sub in alldirs for s in sub if 'AllPlots' in s][0]
kplots = [i for sub in allfiles for i in sub if ('AllPlots' in i) and (i.endswith('.shp'))][0]

# Import as a pandas geodatabase
kplots_gpdf = gpd.read_file(os.path.join(sf_dir, apdir, kplots))

In [48]:
# verify the geodatabase loads as expected
kplots_gpdf.head()

<Projected CRS: EPSG:32613>
Name: WGS 84 / UTM zone 13N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - N hemisphere - 108°W to 102°W - by country
- bounds: (-108.0, 0.0, -102.0, 84.0)
Coordinate Operation:
- name: UTM zone 13N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
# Import external list of plot names, short-form IDs and center point coordinates
coords = pd.read_csv(os.sep.join([source_dir, 'Kueppers_PlotIDs_Coords.csv']))
coords

,SFA_ID,Original_Name,Short,Latitude,Longitude
0,XX-CAR1,Carbon 1,Carbon1,38.824003,-107.021400
1,XX-CAR2,Carbon 2,Carbon2,38.805835,-107.050785
2,CC-UC2,Ohio Pass 1,Ohio1,38.829368,-107.082607
3,XX-PLN1,Point Lookout 1,PL1,38.860497,-106.869908
4,XX-PLN2,Point Lookout 2,PL2,38.853734,-106.869228
5,ER-APL1,Schofield 19,Scho19,38.977070,-106.992279
6,ER-BME1,Schofield 23,Scho23,38.976018,-107.030406
7,ER-GT1,Schofield 24,Scho24,38.977226,-107.005961
8,ER-BME2,Schofield 4,Scho4,38.983970,-107.017749
9,ER-APU1,Schofield 5,Scho5,38.979139,-106.980358


In [9]:
# Create a new field in the kplots geodataframe and store temporary empty values
kplots_gpdf['Plot_ID'] = None

In [10]:
# inner join coords and kplots_gpdf on equal lat values to populate Plot_ID field with short name for reference
# create temporary merge field 'latjoin'
coords['latjoin'] = round(coords['Latitude'], 4)
kplots_gpdf['latjoin'] = round(kplots_gpdf['GEOMCTR_Y'], 4)

# merge as new gpdf
new_df = kplots_gpdf.merge(coords, how = 'inner', on = 'latjoin')
kplots_gpdf['Plot_ID'] = new_df['SFA_ID']
kplots_gpdf = kplots_gpdf.drop(columns=['latjoin'])
kplots_gpdf

,Id,POLY_AREA,AREA_GEO,CENTROID_X,CENTROID_Y,EXT_MIN_X,EXT_MIN_Y,EXT_MAX_X,EXT_MAX_Y,PERIMETER,PERIM_GEO,GEOMCTR_X,GEOMCTR_Y,Plot_ID,PNT_COUNT,geometry
0,0,1221.776739,1221.863563,327968.771206,4.309996e+06,327949.990483,4.309978e+06,327989.478774,4.310016e+06,140.646309,140.651312,-106.984442,38.922037,SG-SWR1,5.0,"POLYGON ((327988.594 4310009.047, 327984.998 4..."
1,0,1177.413239,1177.591884,337758.617869,4.302957e+06,337736.807200,4.302939e+06,337778.177932,4.302973e+06,139.726001,139.736602,-106.869902,38.860505,XX-PLN1,5.0,"POLYGON ((337778.178 4302938.646, 337739.318 4..."
2,0,1287.171970,1287.196562,321927.062151,4.297227e+06,321908.283631,4.297210e+06,321947.804717,4.297248e+06,144.320193,144.321566,-107.050785,38.805835,XX-CAR2,5.0,"POLYGON ((321942.220 4297244.424, 321946.919 4..."
3,0,1235.951437,1235.963259,320848.986798,4.301036e+06,320829.547759,4.301015e+06,320867.887399,4.301054e+06,141.227379,141.228053,-107.064186,38.839922,CC-UC1,5.0,"POLYGON ((320863.247 4301015.870, 320828.662 4..."
4,0,1410.672586,1410.729972,324398.723984,4.313021e+06,324373.194049,4.312994e+06,324424.847580,4.313048e+06,150.371128,150.374179,-107.026375,38.948571,WG-WGM1,5.0,"POLYGON ((324423.962 4313019.558, 324394.979 4..."
5,0,1944.148344,1944.277445,327423.102519,4.316119e+06,327398.007549,4.316097e+06,327447.832708,4.316140e+06,176.879332,176.885215,-106.992277,38.977069,ER-APL1,5.0,"POLYGON ((327446.948 4316140.152, 327443.430 4..."
6,0,1538.640795,1538.633255,319223.426529,4.299901e+06,319196.384466,4.299874e+06,319249.271857,4.299929e+06,157.109924,157.109542,-107.082607,38.829368,CC-UC2,5.0,"POLYGON ((319248.385 4299896.601, 319217.419 4..."
7,0,1818.032892,1818.309599,337801.998507,4.302205e+06,337775.829576,4.302177e+06,337826.470554,4.302231e+06,171.090087,171.103097,-106.869228,38.853734,XX-PLN2,5.0,"POLYGON ((337825.587 4302220.530, 337818.147 4..."
8,0,1516.847184,1516.910426,324523.481959,4.299187e+06,324496.662428,4.299165e+06,324549.502483,4.299208e+06,157.025256,157.028524,-107.021400,38.824003,XX-CAR1,5.0,"POLYGON ((324548.617 4299202.260, 324536.204 4..."
9,0,1472.609336,1472.711700,327781.177429,4.310932e+06,327759.284432,4.310911e+06,327804.486267,4.310953e+06,153.698276,153.703621,-106.986841,38.930426,SG-NES2,5.0,"POLYGON ((327803.601 4310948.312, 327795.439 4..."


## 4. Pull coordinates and areas for all plots

In [108]:
def pullcoords(plots,sf_dir):
    '''
    Calls plots by name, pulls corner coordinates, center coordinates, and areas, and appends the values to a dataframe
    '''
    sites, areas, centers, lats, lons = [], [], [], [], []
    all_coords = pd.DataFrame()

    for i in range(len(plots)):
        site = plots[i]
        plotpoints = [p for p in os.listdir(sf_dir) if (site in p) and ('Bound_pts' in p) and (p.endswith('.shp'))]
        plotpoints_gpdf = gpd.read_file(os.sep.join([sf_dir, plotpoints[0]]))
        pp = plotpoints_gpdf.to_crs('EPSG:4326')
        area = np.round((kplots_gpdf.loc[kplots_gpdf['PLOT_ID'] == site, 
                            ['AREA_GEO']].squeeze()),2)
        ctr_lon = (kplots_gpdf.loc[kplots_gpdf['PLOT_ID'] == site,
                            ['GEOMCTR_X']].squeeze())
        ctr_lat = (kplots_gpdf.loc[kplots_gpdf['PLOT_ID'] == site,
                            ['GEOMCTR_Y']].squeeze())

        # print(ctrpts)
        c1_lon = pp.geometry.x[0]
        c2_lon = pp.geometry.x[1] 
        c3_lon = pp.geometry.x[2]
        c4_lon = pp.geometry.x[3]
        c1_lat = pp.geometry.y[0]
        c2_lat = pp.geometry.y[1]
        c3_lat = pp.geometry.y[2]
        c4_lat = pp.geometry.y[3]

        # make dataframe
        series = list([c1_lat, c1_lon, c2_lat, c2_lon, c3_lat, c3_lon, c4_lat, c4_lon, ctr_lat, ctr_lon, area])
        all_coords[site] = series
    
    all_coords.index = ['corner1_lat', 'corner1_lon', 'corner2_lat', 'corner2_lon', 'corner3_lat', 'corner3_lon', 'corner4_lat', 'corner4_lon', 'center_lat', 'center_lon', 'area']

    return all_coords

In [109]:
allplots = list(kplots_gpdf['PLOT_ID'])
corndir = os.path.join(sf_dir, 'Corner_Points')
all_plotcoords = pullcoords(allplots, corndir)

In [110]:
# view all plot coordinates and geometric area in alpha order
all_plotcoords = all_plotcoords.reindex(sorted(all_plotcoords.columns), axis=1)
all_plotcoords

,CC-CVN1,CC-CVN2,CC-CVS1,CC-EMN1,CC-UC1,CC-UC2,ER-APL1,ER-APU1,ER-BME1,ER-BME2,...,SG-NES2,SG-NES3,SG-SWR1,SR-PVG1,WG-WGM1,XX-CAR1,XX-CAR2,XX-CAR3,XX-PLN1,XX-PLN2
corner1_lat,38.861276,38.875108,38.860913,38.876479,38.840071,38.829620,38.977250,38.979252,38.976162,38.984154,...,38.930610,38.931070,38.922209,38.954649,38.948597,38.824191,38.806019,38.816844,38.860625,38.853939
corner1_lon,-107.075441,-107.019364,-107.035671,-107.067825,-107.064394,-107.082529,-106.992572,-106.980638,-107.030677,-107.017552,...,-106.987036,-106.983338,-106.984656,-107.080824,-107.026671,-107.021514,-107.050979,-107.039542,-106.870152,-106.869372
corner2_lat,38.861306,38.875001,38.861002,38.876532,38.840083,38.829323,38.977259,38.979371,38.976230,38.983789,...,38.930571,38.931066,38.922148,38.954536,38.948809,38.824134,38.805987,38.816886,38.860638,38.853846
corner2_lon,-107.074998,-107.018932,-107.035204,-107.067408,-107.063973,-107.082307,-106.991997,-106.980268,-107.030186,-107.017516,...,-106.986576,-106.982925,-106.984207,-107.080397,-107.026361,-107.021076,-107.050607,-107.039099,-106.869693,-106.868933
corner3_lat,38.860942,38.874653,38.860658,38.876181,38.839733,38.829116,38.976877,38.979037,38.975857,38.983807,...,38.930237,38.930730,38.921870,38.954194,38.948555,38.823804,38.805688,38.816538,38.860332,38.853490
corner3_lon,-107.074954,-107.019033,-107.035108,-107.067311,-107.064006,-107.082658,-106.992027,-106.980063,-107.030175,-107.017972,...,-106.986661,-106.983011,-106.984241,-107.080578,-107.026074,-107.021221,-107.050544,-107.039036,-106.869668,-106.868969
corner4_lat,38.860918,38.874747,38.860547,38.876103,38.839799,38.829408,38.976869,38.978909,38.975811,38.984134,...,38.930266,38.930748,38.921903,38.954313,38.948321,38.823855,38.805672,38.816481,38.860390,38.853587
corner4_lon,-107.075414,-107.019497,-107.035563,-107.067752,-107.064406,-107.082919,-106.992512,-106.980485,-107.030591,-107.017987,...,-106.987089,-106.983497,-106.984656,-107.080997,-107.026402,-107.021670,-107.050999,-107.039505,-106.870117,-106.869495
center_lat,38.861110,38.874876,38.860780,38.876321,38.839922,38.829368,38.977069,38.979139,38.976018,38.983970,...,38.930426,38.930899,38.922037,38.954423,38.948571,38.823996,38.805835,38.816686,38.860505,38.853711
center_lon,-107.075201,-107.019209,-107.035389,-107.067575,-107.064186,-107.082607,-106.992277,-106.980358,-107.030406,-107.017749,...,-106.986841,-106.983193,-106.984442,-107.080698,-107.026375,-107.021371,-107.050785,-107.039297,-106.869902,-106.869195


## 5. Transform dataframe for LBL SFA
Update names and transpose the dataframe to match LBL Watershed Function SFA metadata format

In [111]:
# update names
apc = all_plotcoords

# transpose the dataframe for 'tidy' data format
apc = apc.transpose()

In [112]:
# view the transposed dataframe
apc

,corner1_lat,corner1_lon,corner2_lat,corner2_lon,corner3_lat,corner3_lon,corner4_lat,corner4_lon,center_lat,center_lon,area
CC-CVN1,38.861276,-107.075441,38.861306,-107.074998,38.860942,-107.074954,38.860918,-107.075414,38.861110,-107.075201,1580.03
CC-CVN2,38.875108,-107.019364,38.875001,-107.018932,38.874653,-107.019033,38.874747,-107.019497,38.874876,-107.019209,1645.56
CC-CVS1,38.860913,-107.035671,38.861002,-107.035204,38.860658,-107.035108,38.860547,-107.035563,38.860780,-107.035389,1672.70
CC-EMN1,38.876479,-107.067825,38.876532,-107.067408,38.876181,-107.067311,38.876103,-107.067752,38.876321,-107.067575,1556.87
CC-UC1,38.840071,-107.064394,38.840083,-107.063973,38.839733,-107.064006,38.839799,-107.064406,38.839922,-107.064186,1235.96
CC-UC2,38.829620,-107.082529,38.829323,-107.082307,38.829116,-107.082658,38.829408,-107.082919,38.829368,-107.082607,1538.63
ER-APL1,38.977250,-106.992572,38.977259,-106.991997,38.976877,-106.992027,38.976869,-106.992512,38.977069,-106.992277,1944.28
ER-APU1,38.979252,-106.980638,38.979371,-106.980268,38.979037,-106.980063,38.978909,-106.980485,38.979139,-106.980358,1501.92
ER-BME1,38.976162,-107.030677,38.976230,-107.030186,38.975857,-107.030175,38.975811,-107.030591,38.976018,-107.030406,1605.70
ER-BME2,38.984154,-107.017552,38.983789,-107.017516,38.983807,-107.017972,38.984134,-107.017987,38.983970,-107.017749,1483.96


In [115]:
# export the transposed apc dataframe for plot location registration with LBL Watershed Function SFA
apc.to_csv(os.sep.join([out_dir, 'Kueppers_EastRiver_Plot_Coordinates_Areas_2020_SFAReg.csv']))

## 6. Optional: pull coordinates for a single site

In [42]:
# define the 
cornerpts_dir = os.sep.join([out_dir, 'Kueppers_EastRiver_Plot_Shapefiles_2020_WGS84UTM13N', 'Corner_Points'])

# define the plot name
plot = 'XX-CAR1'

# select the plot's corner points shapefile from the directory containing all plot shapefiles
plotpoints = [i for i in os.listdir(cornerpts_dir) if (plot in i) and ('pts') in i and (i.endswith('.shp'))]

# create a geodataframe from the plot points shapefile
plotpoints_gpdf = gpd.read_file(cornerpts_dir + os.sep + plotpoints[0])

# reproject to WSG 1984, a geographic coordinate system defining points in terms of location on geoid with WGS84 datum
pp = plotpoints_gpdf.to_crs('EPSG:4326')

In [44]:
# get the plot center points stored in the shapefile in WGS 1984 geographic projection
areactr = (kplots_gpdf.loc[kplots_gpdf['Plot_ID'] == plot, 
                            ['Plot_ID', 'AREA_GEO', 'GEOMCTR_X', 'GEOMCTR_Y']])

In [45]:
# pull the center point coordinates and area value
areactr_vals = pd.melt(areactr).sort_values('variable', ascending = False)
areactr_vals

,variable,value
0,Plot_ID,XX-CAR1
3,GEOMCTR_Y,38.824
2,GEOMCTR_X,-107.021
1,AREA_GEO,1516.91
